In [1]:
!pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [2]:
import os
import json
from getpass import getpass

In [3]:
os.environ['OPENAI_API_KEY'] = getpass()

··········


In [4]:
from openai import OpenAI

client = OpenAI()

In [5]:
with open("math_test.json") as f:
    data = json.load(f)

In [6]:
prime = [
    ("Một xưởng may trong tuần thứ nhất thực hiện được 3/8 kế hoạch tháng, tuần thứ hai thực hiện được 3/16 kế hoạch, trong tuần thứ ba thực hiện được 1/3 kế hoạch. Để hoàn thành kế hoạch của tháng thì trong tuần cuối xưởng phải thực hiện bao nhiêu phần kế hoạch?\nA. 5/48\nB. 43/48\nC. 11/48\nD. 27/48", "Reason: Để hoàn thành kế hoạch của tháng thì trong tuần cuối xưởng phải thực hiện:\n1 − ( 3/8 + 3/16 + 1/3) = 5/48 (kế hoạch)\n\nAnswer: A. 5/48"),
    ("Mệnh đề nào sau đây là phủ định của mệnh đề 'Mọi động vật đều di chuyển'?A. Mọi động vật đều không di chuyển\nB. Mọi động vật đều đứng yên\nC. Có ít nhất một động vật không di chuyển\nD. Có ít nhất một động vật di chuyển", "Reason: Mệnh đề \"Có ít nhất một động vật không di chuyển\" là phủ định của mệnh đề \"Mọi động vật đều di chuyển\"\n\nAnswer: C. Có ít nhất một động vật không di chuyển"),
    ("Cho các đoạn thẳng AB = 4cm; MN = 5cm; EF = 3cm; PQ = 8cm; IK = 7cm . Sắp xếp độ dài các đoạn thẳng theo thứ tự tăng dần?\nA. EF, AB, MN, IK, PQ\nB. PQ, IK, MN, AB, EF\nC. EF, AB, IK, PQ, MN\nD. EF, MN, IK, PQ, AB", "Reason: Độ dài các đoạn thẳng theo thứ tự tăng dần là: EF, AB, MN, IK, PQ\n\nAnswer: A. EF, AB, MN, IK, PQ")
]

In [7]:
outputs = []
chat_completions = []

In [8]:
for idx, sample in enumerate(data['data']):
    message = sample["question"]
    for choice in sample["choices"]:
        message += f"\n{choice}"
    message += f"\nĐáp án: "

    print(f"At datapoint {idx}\n")
    print(message,"\n")

    response = client.chat.completions.create(
        model='gpt-3.5-turbo-16k',
        messages=[
            {"role": "system", "content": "You are a helpful assistant that solves Vietnamese word math problem in the form of multi-choice questions, specifically you have to choose the one correct option among multiple options. You solve the question step by step and give explanation along with the answer.  Your reply must be in the form of \"Reason: {your_reasoning_step_by_step}\n\nAnswer: {your_chosen_option}\"."},
            {"role": "system", "name":"example_user", "content": prime[0][0]},
            {"role": "system", "name": "example_assistant", "content": prime[0][1]},
            {"role": "system", "name":"example_user", "content": prime[1][0]},
            {"role": "system", "name": "example_assistant", "content": prime[1][1]},
            {"role": "system", "name":"example_user", "content": prime[2][0]},
            {"role": "system", "name": "example_assistant", "content": prime[2][1]},
            {"role": "user", "content": message},
        ],
        temperature=0,
    )

    chat_completions.append(response)
    output = {}
    output["answer"] = response.choices[0].message.content
    output["id"] = sample["id"]
    print(output["answer"], "\n")
    print("=================================================\n")
    outputs.append(output)


At datapoint 0

Một cửa hàng đã bán 30% số hàng hiện có và thu được 15 000 000 đồng. Hỏi nếu bán hết hàng thì cửa hàng thu được bao nhiêu tiền?
A. 4 500 000 đồng
B. 45 000 000 đồng
C. 50 000 000 đồng
D. 450 000 000 đồng
Đáp án:  

Reason: Ta có thể sử dụng tỷ lệ để tính toán. Giả sử số hàng hiện có là x.

30% số hàng hiện có = 15 000 000 đồng

Từ đó, ta có phương trình: 0.3x = 15 000 000

Giải phương trình, ta có: x = 15 000 000 / 0.3 = 50 000 000

Vậy nếu bán hết hàng, cửa hàng sẽ thu được 50 000 000 đồng.

Answer: C. 50 000 000 đồng 


At datapoint 1

Một người đi xe đạp từ A lúc 7 giờ với vận tốc 12km/h. Đến 8 giờ một người đi xe máy cũng từ A đuổi theo người đi xe đạp với vận tốc 42km/h. Hỏi người đi xe máy đuổi kịp người đi xe đạp lúc mấy giờ?
A. 24 phút
B. 1 giờ
C. 7 giờ 24 phút
D. 8 giờ 24 phút
Đáp án:  



RateLimitError: ignored

In [ ]:
outputs

In [ ]:
with open('cot_few_shot_predict.json', 'w', encoding="utf-8") as f:
    json.dump(outputs, f, indent=4, ensure_ascii=False)

In [ ]:
import pickle
with open("bin_cot_few_shot_submit.dat", "wb") as f:
    pickle.dump(chat_completions, f)